In [43]:
%pip install numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from typing import Tuple, List, Union, Dict
from collections import defaultdict
import numpy as np
import re
import math

In [2]:
def number_string_to_int_list(s: str) -> List[int]:
    return [int(c) for c in s.split(" ") if c.lstrip(" ")]

# Day 1

In [46]:

def first_digit_char(s: str) -> Tuple[int, int]:
    for i, c in enumerate(s):
        if c.isdigit():
            return i,int(c)
    raise Exception(f"No digit found in {s}")

def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        number = int(f"{first_digit_char(line)[1]}{first_digit_char(reversed(line))[1]}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)
        
print("Total sum is", total_sum)

Total sum is 55834


## Part 2

In [47]:
digits = ("one", "two", "three", "four", "five", "six", "seven", "eight", "nine")

def first_digit_word(s: str, digits: List[str]) -> Tuple[int,int]:
    min_pair = (len(s), None)
    for i,d in enumerate(digits):
        idx = s.find(d)
        if idx != -1 and idx < min_pair[0]:
            min_pair = (idx, i+1)
    return min_pair


def solve(lines) -> int:
    total_sum = 0
    for line in lines:
        _, d1 = min(first_digit_char(line),first_digit_word(line, digits), key=lambda t: t[0])

        line_reversed = line[::-1]
        digits_reversed = [d[::-1] for d in digits]
        _, d2 = min(first_digit_char(line_reversed),first_digit_word(line_reversed, digits_reversed), key=lambda t: t[0])

        number = int(f"{d1}{d2}")
        total_sum += number
    return total_sum

with open("input_day_1") as f:
    total_sum = solve(f)

print("Total Sum", total_sum)

Total Sum 53221


# Day 2

In [48]:


def game_string_to_dict(game_str: str) -> dict[str, int]:
    game, sets = game_str.split(": ")
    game_id = int(game.split(" ")[1])
    sets = sets.split("; ")

    game_stats = defaultdict(int)
    game_stats["Game_ID"] = game_id

    for set_string in sets:
        for color in set_string.split(", "):
            v,k = color.split(" ")
            assert k in ("red", "green", "blue"), f"not in colors '{k}'"
            game_stats[k] = max(int(v), game_stats[k])
    return game_stats

id_sum = 0
with open("input_day_2") as f:
    for r in f:
        r = r.rstrip("\n")
        game = game_string_to_dict(r)
        if game["red"] <= 12 and game["green"] <= 13 and game["blue"] <= 14:
            id_sum += game["Game_ID"]
print("Total sum of filtered Game IDs", id_sum)    

Total sum of filtered Game IDs 2348


In [49]:
from functools import reduce
from operator import mul


prod_sum = 0
with open("input_day_2") as f:
    for r in f:
        r = r.rstrip("\n")
        game = game_string_to_dict(r)
        cubes = [c for c in [game["red"], game["blue"], game["green"]] if c > 0]
        prod_sum += reduce(mul, cubes, 1)
print("Total sum of products of games", prod_sum)

Total sum of products of games 76008


# Day 3

In [50]:
def parse_numbers_and_symbol_indezes(s: str) -> Tuple[Dict[Tuple[int, int], str], Dict[Tuple[int, int], str]]:
    numbers_indezes = dict() # key is inex (row, end_col), value is number
    symbol_indezes = dict() # key is index (row, col), value is symbol

    for row_idx, row in enumerate(s):
        current_number_seq = ""
        row = row.rstrip("\n")
        for i, c in enumerate(row):
            if c.isdigit():
                current_number_seq += c
            else:
                if current_number_seq:
                    numbers_indezes[(row_idx, i-1)]=current_number_seq
                    current_number_seq = ""
                if c != ".":
                    symbol_indezes[(row_idx, i)] = c
        if current_number_seq:
            numbers_indezes[(row_idx, i)]=current_number_seq
    return numbers_indezes, symbol_indezes

def compute_neighbour_symbols(numbers_indezes: Dict[Tuple[int, int], str], symbol_indezes: Dict[Tuple[int, int], str]) -> Tuple[Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]], Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]]:
    numbers_with_neighbour_symbols: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]] = dict()
    symbols_with_neighbour_numbers: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]] = {
        idxs: (symbol, dict()) for idxs, symbol in symbol_indezes.items()
    }
    for (row_idx, end_index), num in sorted(numbers_indezes.items(), key=lambda t: t[0]):
        # neighbours are indezes 1 cells around the number, including diagonals
        # note that the number begins at end_index-len(num)+1
        neighbours = [(row_idx, end_index-len(num)), (row_idx, end_index+1)]
        for ic in (row_idx-1, row_idx+1):
            for j in range(end_index-len(num), end_index+2):
                neighbours.append((ic,j))

        numbers_with_neighbour_symbols[(row_idx, end_index)] = (num, dict())
        for ic,j in neighbours:
            if (ic,j) in symbol_indezes:
                numbers_with_neighbour_symbols[(row_idx, end_index)][1][(ic,j)] = symbol_indezes[(ic,j)]
                symbols_with_neighbour_numbers[(ic,j)][1][(row_idx, end_index)] = num
    return numbers_with_neighbour_symbols, symbols_with_neighbour_numbers


def compute_product_number_sum(numbers_with_neighbour_symbols: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]) -> int:
    product_num_sum = 0
    for (row_idx, end_index), (num, symbols) in sorted(numbers_with_neighbour_symbols.items(), key=lambda t: t[0]):
        if len(symbols):
            print("Valid serial number", num, "at", row_idx, end_index-len(num)+1)
            product_num_sum += int(num)
        # else:
        #     print(f"No symbol found for number {num} at {(row_idx, end_index)}")
    return product_num_sum

def compute_gear_ratio_sum(symbols_with_neighbour_numbers: Dict[Tuple[int, int], Tuple[str, Dict[Tuple[int, int], str]]]) -> int:
    gear_ratio_sum = 0
    for (row_idx, col_idx), (symbol, numbers) in sorted(symbols_with_neighbour_numbers.items(), key=lambda t: t[0]):
        # if symbol is '*' and has exactly 2 neighbouring numbers, compute the gear ratio
        if symbol == "*" and len(numbers) == 2:
            gear_ratio = 1
            for num in numbers.values():
                gear_ratio *= int(num)
            gear_ratio_sum += gear_ratio
    return gear_ratio_sum

with open("input_day_3") as f:
    numbers_indezes, symbol_indezes = parse_numbers_and_symbol_indezes(f)
    numbers_with_neighbour_symbols, symbols_with_neighbour_numbers = compute_neighbour_symbols(numbers_indezes, symbol_indezes)
    print("Sum of products of numbers", compute_product_number_sum(numbers_with_neighbour_symbols))
    print("Sum of gear ratios", compute_gear_ratio_sum(symbols_with_neighbour_numbers))

Valid serial number 577 at 0 25
Valid serial number 56 at 0 90
Valid serial number 446 at 0 102
Valid serial number 793 at 0 106
Valid serial number 627 at 1 52
Valid serial number 623 at 1 76
Valid serial number 610 at 1 85
Valid serial number 16 at 1 124
Valid serial number 891 at 1 132
Valid serial number 336 at 2 21
Valid serial number 470 at 2 31
Valid serial number 84 at 2 57
Valid serial number 34 at 2 70
Valid serial number 76 at 2 95
Valid serial number 117 at 3 1
Valid serial number 359 at 3 7
Valid serial number 595 at 3 27
Valid serial number 129 at 3 42
Valid serial number 963 at 3 47
Valid serial number 722 at 3 53
Valid serial number 128 at 3 66
Valid serial number 192 at 3 77
Valid serial number 313 at 3 81
Valid serial number 31 at 3 92
Valid serial number 887 at 3 102
Valid serial number 234 at 3 120
Valid serial number 298 at 4 12
Valid serial number 922 at 4 20
Valid serial number 482 at 4 34
Valid serial number 395 at 4 97
Valid serial number 166 at 4 131
Valid ser

In [51]:
with open("input_day_3") as f:
    arr = f.readlines()

def is_symbol(row, col) -> bool:
    # check bounds
    if row < 0 or row >= len(arr) or col < 0 or col >= len(arr[row]) -1:
        return False
    # must be not digit or dot
    if not arr[row][col].isdigit() and arr[row][col] != ".":
        return True

def has_symbol_neighbour(row, col) -> bool:
    # check neighbours
    for i in range(row-1, row+2):
        for j in range(col-1, col+2):
            if is_symbol(i,j):
                return True
    return False

# find all numbers in the array
prod_sum = 0
for ir,r in enumerate(arr):
    number_seq = ""
    found_neighbour_symbol = False
    r = r.rstrip("\n")
    for ic, c in enumerate(r):
        if c.isdigit():
            number_seq += c
            if has_symbol_neighbour(ir, ic):
                found_neighbour_symbol = True
        elif number_seq:
            # terminate number sequence
            if found_neighbour_symbol:
                prod_sum += int(number_seq)
                print("Valid serial number", number_seq, "at", ir, ic-len(number_seq))
                found_neighbour_symbol = False
            # else: 
            #     print("Did not find symbol neighbour for number", number_seq, "at", (ir, ic-len(number_seq)))
            number_seq = ""
    # at end of row, check if number sequence is valid
    if number_seq and found_neighbour_symbol:
        print("Valid serial number", number_seq, "at", ir, ic-len(number_seq))
        prod_sum += int(number_seq)

print("Sum of products of numbers", prod_sum)


Valid serial number 577 at 0 25
Valid serial number 56 at 0 90
Valid serial number 446 at 0 102
Valid serial number 793 at 0 106
Valid serial number 627 at 1 52
Valid serial number 623 at 1 76
Valid serial number 610 at 1 85
Valid serial number 16 at 1 124
Valid serial number 891 at 1 132
Valid serial number 336 at 2 21
Valid serial number 470 at 2 31
Valid serial number 84 at 2 57
Valid serial number 34 at 2 70
Valid serial number 76 at 2 95
Valid serial number 117 at 3 1
Valid serial number 359 at 3 7
Valid serial number 595 at 3 27
Valid serial number 129 at 3 42
Valid serial number 963 at 3 47
Valid serial number 722 at 3 53
Valid serial number 128 at 3 66
Valid serial number 192 at 3 77
Valid serial number 313 at 3 81
Valid serial number 31 at 3 92
Valid serial number 887 at 3 102
Valid serial number 234 at 3 120
Valid serial number 298 at 4 12
Valid serial number 922 at 4 20
Valid serial number 482 at 4 34
Valid serial number 395 at 4 97
Valid serial number 166 at 4 131
Valid ser

# Day 4

In [52]:
def parse_card(card_line: str) -> int:
    segments = line.rstrip("\n").split(": ")[1].split(" | ")
    winning_numbers, my_numbers = [set([int(a) for a in s.split(" ") if a]) for s in segments]
    matching_numbers = winning_numbers.intersection(my_numbers)
    return len(matching_numbers)

score = 0
with open("input_day_4") as f:
    for line in f:
        matches = parse_card(line)
        if matches:
            score += pow(2, matches-1)
        
        
print("Score", score)

Score 20829


In [53]:
num_scratchcards = 0
multipliers = []
with open("input_day_4") as f:
    for line in f:
        matches = parse_card(line)

        card_count = (multipliers.pop(0)+1) if multipliers else 1
        num_scratchcards += card_count

        # increase next matches elements in the queue by multiplier
        # append if nessesary
        for i in range(matches):
            if i < len(multipliers):
                multipliers[i] += card_count
            else:
                multipliers.append(card_count)

        

assert len(multipliers) == 0, f"Queue should be empty {multipliers}"
print("Total number of scratchcards", num_scratchcards)

Total number of scratchcards 12648035


# Day 5

In [54]:
class ConversionDict():
    def __init__(self):
        self.mapping = dict()

    def add_conversion(self, dest: int, src: int, length: int):
        self.mapping[src] = (dest, length)

    def __getitem__(self, __key: Tuple[int,int]) -> List[Tuple[int,int]]:
        start, length = __key

        # if length == 1:
        if False:
            for src, (dest, conv_length) in self.mapping.items():
                if start >= src and start < src+conv_length:
                    return [(dest + (start-src), 1)]
            return [(start, 1)]
        else:
            resulting_ranges = []
            for src, (dest, conv_length) in sorted(self.mapping.items(), key=lambda t: t[0]):
                if start < src: 
                    # this conversion is after the start, so we can add the range up to the conversion
                    offset = min(length, src-start)
                    assert offset >= 0, f"Offset should be positive {offset}"
                    resulting_ranges.append((start, offset))
                    start += offset
                    length -= offset

                if length == 0:
                    return resulting_ranges
                    
                # add part of the conversion that is covered by the range
                if start >= src and start < src+conv_length:
                    offset_in_conv = min(start-src, conv_length)
                    assert offset_in_conv >= 0, f"Offset in conversion should be positive {offset_in_conv}"
                    assert offset_in_conv <= conv_length, f"Offset in conversion should be smaller than conversion length {offset_in_conv} < {conv_length}"
                    covered_length = min(length, conv_length-offset_in_conv)
                    assert covered_length >= 0, f"Covered length should be positive {covered_length}"
                    resulting_ranges.append((dest+offset_in_conv, covered_length))
                    start += covered_length
                    length -= covered_length

                    if length == 0:
                        return resulting_ranges
            assert length > 0, f"Length should be positive {length}"
            resulting_ranges.append((start, length))
            return resulting_ranges


def parse_map(map_string: str):
    components = map_string.split("\n")
    mappipng = components[0].split(" ")[0]
    src = mappipng.split("-")[0]
    dest = mappipng.split("-")[2]
    mapping = ConversionDict()

    for line in components[1:]:
        numbers = number_string_to_int_list(line)
        mapping.add_conversion(*numbers)
    return src, dest, mapping

with open("input_day_5_test1") as f:
    seeds = number_string_to_int_list(f.readline().rstrip("\n").split(": ")[1])
    seeds = [(s, 1) for s in seeds]
    print(seeds)
    _ = f.readline()
    maps = [parse_map(m) for m in f.read().split("\n\n")]
    print(maps)

for m in maps:
    new_seeds = []
    for s in seeds:
        new_seeds += m[2][s]
    seeds = new_seeds
print("Seeds after final conversion", seeds)
print("Min location", min(seeds)[0])

[(79, 1), (14, 1), (55, 1), (13, 1)]
[('seed', 'soil', <__main__.ConversionDict object at 0x7f9d2cd67590>), ('soil', 'fertilizer', <__main__.ConversionDict object at 0x7f9d2cd64210>), ('fertilizer', 'water', <__main__.ConversionDict object at 0x7f9d2cd641d0>), ('water', 'light', <__main__.ConversionDict object at 0x7f9d2cd65110>), ('light', 'temperature', <__main__.ConversionDict object at 0x7f9d2eff4450>), ('temperature', 'humidity', <__main__.ConversionDict object at 0x7f9d2eff6210>), ('humidity', 'location', <__main__.ConversionDict object at 0x7f9d2eff7dd0>)]
Seeds after final conversion [(82, 1), (43, 1), (86, 1), (35, 1)]
Min location 35


In [55]:
with open("input_day_5") as f:
    seeds = number_string_to_int_list(f.readline().rstrip("\n").split(": ")[1])
    # convert pair of seeds to (start, range_length)
    seeds = [(s, seeds[i+1]) for i,s in enumerate(seeds) if i%2 == 0]
    print(seeds)
    
    _ = f.readline()

    maps = [parse_map(m) for m in f.read().split("\n\n")]
    print(maps)

for m in maps:
    new_seeds = []
    for s in seeds:
        new_seeds += m[2][s]
    seeds = new_seeds
print("Seeds after final conversion", seeds)
print("Min location", min(seeds))

[(1482445116, 339187393), (3210489476, 511905836), (42566461, 51849137), (256584102, 379575844), (3040181568, 139966026), (4018529087, 116808249), (2887351536, 89515778), (669731009, 806888490), (2369242654, 489923931), (2086168596, 82891253)]
[('seed', 'soil', <__main__.ConversionDict object at 0x7f9d2cd63290>), ('soil', 'fertilizer', <__main__.ConversionDict object at 0x7f9d2cd63410>), ('fertilizer', 'water', <__main__.ConversionDict object at 0x7f9d2cd61390>), ('water', 'light', <__main__.ConversionDict object at 0x7f9d2cd61f10>), ('light', 'temperature', <__main__.ConversionDict object at 0x7f9d2cd62b50>), ('temperature', 'humidity', <__main__.ConversionDict object at 0x7f9d2cd61d10>), ('humidity', 'location', <__main__.ConversionDict object at 0x7f9d2cd60b50>)]
Seeds after final conversion [(3134759866, 20211188), (3154971054, 23987553), (718111107, 2523466), (3577569390, 6194161), (906358348, 15352358), (1009711192, 17989949), (47236480, 14645668), (3231718897, 1566255), (2239990

# Day 6

In [56]:
# parse input
# Time:      7  15   30
# Distance:  9  40  200

with open("input_day_6") as f:
    time = number_string_to_int_list(f.readline().rstrip("\n").split(": ")[1])
    distance = number_string_to_int_list(f.readline().rstrip("\n").split(": ")[1])

def compute_race_winning_posibilites(t: int, d: int) -> int:
    # solve: d < (t-p)*p
    # for p
    # p^2 - t*p + d < 0
    # solving for p
    # p = (t +- sqrt(t^2 - 4*d))/2
    p1 = (t + np.sqrt(t**2 - 4*d))/2
    p2 = (t - np.sqrt(t**2 - 4*d))/2
    x = math.ceil(p1) - math.floor(p2) -1
    print(p1,p2,x)
    # possible_durations = [p for p in range(1, t+1) if d < (t-p)*p]
    # x = len(possible_durations)
    return x
# compute possible duration of button presses to reach a higher distance for each race
product = 1
for t,d in zip(time,distance):
    product *= compute_race_winning_posibilites(t,d)
print("Possible durations product", product)
assert product == 633080

26.219544457292887 7.780455542707113 19
62.663521732655695 27.336478267344305 35
72.25337817275583 16.746621827244166 56
51.30662386291807 34.69337613708193 17
Possible durations product 633080


In [57]:
with open("input_day_6") as f:
    time = int(f.readline().rstrip("\n").split(": ")[1].replace(" ", ""))
    distance = int(f.readline().rstrip("\n").split(": ")[1].replace(" ", ""))

print("Time", time, "Distance", distance)
x = compute_race_winning_posibilites(time, distance)
print("Possible durations", x)
assert x == 20048741

Time 34908986 Distance 204171312101780
27478863.992000893 7430122.007999105 20048741
Possible durations 20048741


# Day 7

In [81]:
from collections import Counter
from functools import cmp_to_key, partial


def compare_hands(order: List[str], joker: bool, hand1: str, hand2: str) -> int:
    
    print("Hands", hand1, hand2)
    # compare by rules
    g1 = Counter(hand1)
    g2 = Counter(hand2)
    print("Counters", g1, g2)

    # five of a kind 1
    # four of a kind 2
    # full house 2
    # three of a kind 3
    # two pairs 3
    # one pair 4

    if joker:
        if "J" in g1 and len(g1) > 1:
            j_count = g1["J"]
            del g1["J"]
            g1[max(g1, key=g1.get)] += j_count
        if "J" in g2 and len(g2) > 1:
            j_count = g2["J"]
            del g2["J"]
            g2[max(g2, key=g2.get)] += j_count

    assert sum(g1.values()) == 5, f"Hand 1 {hand1} should have 5 cards {g1}"
    assert sum(g2.values()) == 5, f"Hand 2 {hand2} should have 5 cards {g2}"

    if len(g1) < len(g2):
        print("Hand 1 wins")
        return 1
    elif len(g1) > len(g2):
        print("Hand 2 wins")
        return -1

    if len(g1) == len(g2) == 2:
        # four of a kind
        if 4 in g1.values() and 4 not in g2.values():
            print("Hand 1 wins")
            return 1
        elif 4 in g2.values() and 4 not in g1.values():
            print("Hand 2 wins")
            return -1
        # full house, proceed with by card comparision
        
    elif len(g1) == len(g2) == 3:
        # three of a kind
        if 3 in g1.values() and 3 not in g2.values():
            print("Hand 1 wins")
            return 1
        elif 3 in g2.values() and 3 not in g1.values():
            print("Hand 2 wins")
            return -1
        # two pairs, proceed with by card comparision
        
    # compare by character
    for c1,c2 in zip(hand1, hand2):
        if c1 != c2:
            i1 = order.index(c1)
            i2 = order.index(c2)
            assert i1 >= 0 and i2 >= 0, f"Character not in order {c1} {c2}"
            print("Hand", 1 if i1 < i2 else 2, "wins by card comparison")
            return 1 if i1 < i2 else -1
        
    assert False, "All cards should be different"
    return 0


with open("input_day_7") as f:
    lines = f.readlines()
    hands_with_bid = [l.rstrip("\n").split(" ") for l in lines]
    # print(hands_with_bid)

# sort hands 
order = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]
hands_with_bid = sorted(hands_with_bid, key=lambda x: cmp_to_key(partial(compare_hands, order, False))(x[0]))
# print(hands_with_bid)
# assert "QQQJA" == hands_with_bid[4][0], f"Expected QQQJA, got {hands_with_bid[4][0]}"
# assert "T55J5" == hands_with_bid[3][0], f"Expected T55J5, got {hands_with_bid[3][0]}"
# assert "KK677" == hands_with_bid[2][0], f"Expected KK677, got {hands_with_bid[2][0]}"
# assert "KTJJT" == hands_with_bid[1][0], f"Expected KTJJT, got {hands_with_bid[1][0]}"
# assert "32T3K" == hands_with_bid[0][0], f"Expected 32T3K, got {hands_with_bid[0][0]}"

bid_rank_product = 0
for i, (hand, bid) in enumerate(hands_with_bid):
    # print(f"Hand {i+1} with bid {bid} has hand {hand}")
    bid_rank_product += (i+1)*int(bid)
print("Bid rank product", bid_rank_product)

Hands Q33J3 A833A
Counters Counter({'3': 3, 'Q': 1, 'J': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 1 wins
Hands 55KK5 Q33J3
Counters Counter({'5': 3, 'K': 2}) Counter({'3': 3, 'Q': 1, 'J': 1})
Hand 1 wins
Hands K4457 55KK5
Counters Counter({'4': 2, 'K': 1, '5': 1, '7': 1}) Counter({'5': 3, 'K': 2})
Hand 2 wins
Hands K4457 Q33J3
Counters Counter({'4': 2, 'K': 1, '5': 1, '7': 1}) Counter({'3': 3, 'Q': 1, 'J': 1})
Hand 2 wins
Hands K4457 A833A
Counters Counter({'4': 2, 'K': 1, '5': 1, '7': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 2 wins
Hands Q3QT3 Q33J3
Counters Counter({'Q': 2, '3': 2, 'T': 1}) Counter({'3': 3, 'Q': 1, 'J': 1})
Hand 2 wins
Hands Q3QT3 A833A
Counters Counter({'Q': 2, '3': 2, 'T': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 2 wins by card comparison
Hands Q3QT3 K4457
Counters Counter({'Q': 2, '3': 2, 'T': 1}) Counter({'4': 2, 'K': 1, '5': 1, '7': 1})
Hand 1 wins
Hands QJK32 A833A
Counters Counter({'Q': 1, 'J': 1, 'K': 1, '3': 1, '2': 1}) Counter({'A': 2, '3': 2, '8': 1}

In [83]:
with open("input_day_7") as f:
    lines = f.readlines()
    hands_with_bid = [l.rstrip("\n").split(" ") for l in lines]
    # print(hands_with_bid)


# sort hands 
order = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]

two_pair = "23J5J"
three_of_a_kind = "JAAKA"
compare_hands(order, True, two_pair, three_of_a_kind)
hands_with_bid = sorted(hands_with_bid, key=lambda x: cmp_to_key(partial(compare_hands, order, True))(x[0]))
print(hands_with_bid)
# assert "AAAAJ" == hands_with_bid[6][0], f"Expected AAAAJ, got {hands_with_bid[6][0]}"
# assert "JJJJJ" == hands_with_bid[5][0], f"Expected JJJJJ, got {hands_with_bid[5][0]}"
# assert "KTJJT" == hands_with_bid[4][0], f"Expected KTJJT, got {hands_with_bid[4][0]}"
# assert "QQQJA" == hands_with_bid[3][0], f"Expected QQQJA, got {hands_with_bid[3][0]}"
# assert "T55J5" == hands_with_bid[2][0], f"Expected T55J5, got {hands_with_bid[2][0]}"
# assert "KK677" == hands_with_bid[1][0], f"Expected KK677, got {hands_with_bid[1][0]}"
# assert "32T3K" == hands_with_bid[0][0], f"Expected 32T3K, got {hands_with_bid[0][0]}"

bid_rank_product = 0
for i, (hand, bid) in enumerate(hands_with_bid):
    print(f"Hand {i+1} with bid {bid} has hand {hand}")
    bid_rank_product += (i+1)*int(bid)
print("Bid rank product", bid_rank_product)

Hands 23J5J JAAKA
Counters Counter({'J': 2, '2': 1, '3': 1, '5': 1}) Counter({'A': 3, 'J': 1, 'K': 1})
Hand 2 wins
Hands Q33J3 A833A
Counters Counter({'3': 3, 'Q': 1, 'J': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 1 wins
Hands 55KK5 Q33J3
Counters Counter({'5': 3, 'K': 2}) Counter({'3': 3, 'Q': 1, 'J': 1})
Hand 2 wins
Hands 55KK5 Q33J3
Counters Counter({'5': 3, 'K': 2}) Counter({'3': 3, 'Q': 1, 'J': 1})
Hand 2 wins
Hands 55KK5 A833A
Counters Counter({'5': 3, 'K': 2}) Counter({'A': 2, '3': 2, '8': 1})
Hand 1 wins
Hands K4457 55KK5
Counters Counter({'4': 2, 'K': 1, '5': 1, '7': 1}) Counter({'5': 3, 'K': 2})
Hand 2 wins
Hands K4457 A833A
Counters Counter({'4': 2, 'K': 1, '5': 1, '7': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 2 wins
Hands Q3QT3 55KK5
Counters Counter({'Q': 2, '3': 2, 'T': 1}) Counter({'5': 3, 'K': 2})
Hand 2 wins
Hands Q3QT3 A833A
Counters Counter({'Q': 2, '3': 2, 'T': 1}) Counter({'A': 2, '3': 2, '8': 1})
Hand 2 wins by card comparison
Hands Q3QT3 K4457
Counters Counter

# Day 8

In [14]:
with open("input_day_8") as f:
    sequence = f.readline().rstrip("\n")
    print(sequence)
    f.readline()
    mappings = {l.split(" = ")[0]: l.split(" = ")[1][1:-1].split(", ") for l in f.read().split("\n")}
    print(mappings)

steps = 0
position = "AAA"
while position != "ZZZ":
    direction = sequence[steps % len(sequence)]
    direction_index = 0 if direction == "L" else 1
    position = mappings[position][direction_index]
    steps += 1
print("Steps", steps)

LLRLLRRLRLRRRLRRLLRRRLRLRLRRLRRRLRRLRLRLLRLLLRRRLRRLRRRLRRRLRRRLRLRRLLRRLRRLRRLRRRLRLRRRLLRLRRLRRRLRLRRRLRRRLRLRRRLLRRRLRRRLRLRRLRLRRRLLRRLRRLRRLRRLRLRLRRRLLRRRLRRLRRRLRLRLRRRLLRLRRLLRLRRLRLRRRLRLRRLLRRRLLRRLRLRLLRLLRRLRRLLRRLRLRRLRLRLRRRLRRLRLLLLRRLRLRLRRRLLLRRRLRRLRRLRLLRLRRRLLLRRRLRRRLRRRR
{'NQT': ['TXC', 'RVJ'], 'FPT': ['PNS', 'KJL'], 'BNQ': ['THG', 'LCV'], 'SPL': ['VBH', 'NNV'], 'TLM': ['LVQ', 'PGT'], 'GHC': ['XKN', 'SPR'], 'PHT': ['HMF', 'DST'], 'FSF': ['JCM', 'SMT'], 'GDD': ['FHJ', 'RBS'], 'GVR': ['FVD', 'FVD'], 'SST': ['PMF', 'MGC'], 'ZZZ': ['VLV', 'SQV'], 'SDV': ['DBL', 'GTL'], 'XSP': ['GRV', 'RFM'], 'SKG': ['NGH', 'VDX'], 'BPK': ['RMK', 'LCQ'], 'VSC': ['DRN', 'SFR'], 'BFR': ['BJR', 'GMD'], 'HTM': ['PFM', 'LVD'], 'XVP': ['LDP', 'LDP'], 'DRD': ['SCH', 'LKD'], 'DSJ': ['GSQ', 'JTN'], 'VTS': ['BNQ', 'VFX'], 'KVF': ['HTP', 'MQK'], 'GPS': ['FXQ', 'TPF'], 'VRC': ['RTK', 'RTK'], 'HRD': ['PMQ', 'JCR'], 'DJK': ['FTC', 'KXH'], 'VGJ': ['PCJ', 'TVH'], 'QMN': ['TLC', 'HNG'], 'DMH': ['QSD', 

In [25]:
with open("input_day_8_part2") as f:
    sequence = f.readline().rstrip("\n")
    print(sequence)
    f.readline()
    mappings = {l.split(" = ")[0]: l.split(" = ")[1][1:-1].split(", ") for l in f.read().split("\n")}
    print("Total number of mappings", len(mappings))

steps = 0
positions = [k for k in mappings.keys() if k.endswith("A")]
print("Number of simultaneous positions", len(positions))
z_node_seen_at = [defaultdict(list) for _ in range(len(positions))]

periodicity_steps = [None for _ in range(len(positions))]

while not all(p.endswith("Z") for p in positions):
    direction = sequence[steps % len(sequence)]
    direction_index = 0 if direction == "L" else 1

    positions = [mappings[p][direction_index] for p in positions]
    steps += 1


    for i,p in enumerate(positions):
        if p.endswith("Z"):
            z_node_seen_at[i][p].append(steps)
            assert len(z_node_seen_at[i]) <= 1, f"Multiple Z nodes seen at {p} for position {i}"

            if len(z_node_seen_at[i][p]) == 3:
                p_0_1 = z_node_seen_at[i][p][1] - z_node_seen_at[i][p][0]
                p_1_2 = z_node_seen_at[i][p][2] - z_node_seen_at[i][p][1]
                p_0 = z_node_seen_at[i][p][0]
                print("Periodicity for position", i, "is", p_0_1, p_1_2, p_0)
                assert p_0 == p_0_1, f"Periodicity should be the same {p_0} {p_0_1}"
                assert p_0_1 == p_1_2, f"Periodicity should be the same {p_0_1} {p_1_2}"
                periodicity_steps[i] = p_0_1
    
    if all(p for p in periodicity_steps):
        print("Found all periodicity steps", periodicity_steps)
        break

# compute lcm of periodicity steps using numpy
periodicity_steps = np.array(periodicity_steps)
lcm = np.lcm.reduce(periodicity_steps)
print("LCM", lcm)
print("Steps", steps)

LLRLLRRLRLRRRLRRLLRRRLRLRLRRLRRRLRRLRLRLLRLLLRRRLRRLRRRLRRRLRRRLRLRRLLRRLRRLRRLRRRLRLRRRLLRLRRLRRRLRLRRRLRRRLRLRRRLLRRRLRRRLRLRRLRLRRRLLRRLRRLRRLRRLRLRLRRRLLRRRLRRLRRRLRLRLRRRLLRLRRLLRLRRLRLRRRLRLRRLLRRRLLRRLRLRLLRLLRRLRRLLRRLRLRRLRLRLRRRLRRLRLLLLRRLRLRLRRRLLLRRRLRRLRRLRLLRLRRRLLLRRRLRRRLRRRR
Total number of mappings 738
Number of simultaneous positions 6
Periodicity for position 2 is 12599 12599 12599
Periodicity for position 4 is 13771 13771 13771
Periodicity for position 1 is 17287 17287 17287
Periodicity for position 0 is 19631 19631 19631
Periodicity for position 5 is 20803 20803 20803
Periodicity for position 3 is 23147 23147 23147
Found all periodicity steps [19631, 17287, 12599, 23147, 13771, 20803]
LCM 13129439557681
Steps 69441
